# Import libraries 



In [80]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
import os

# Read the Csv file 

In [81]:
df = pd.read_csv('/content/unit_cell_catalog2000.csv')

Classify into first_nodes for first group and sec_node for the second
split the x, y and z nodes

In [82]:
from numpy.core.fromnumeric import shape

# Create a list of all column names
column_names = df.columns.values.tolist()

# Create empty dataframes to store data
nodal_positions = pd.DataFrame()
bar_connectivity = pd.DataFrame()
first_nodes = pd.DataFrame()
second_nodes = pd.DataFrame()
x_nodes = pd.DataFrame()
y_nodes = pd.DataFrame()
z_nodes = pd.DataFrame()

# Iterate the whole dataframe looking for specific rows and sorting them into respective dataframes
for index, column_name in enumerate(column_names):
    # Find the nodal positions first using regex
    if re.search("^nodal_positions", column_name):
        nodal_positions[column_name] = df.iloc[: , index]
        node_num = int(re.search("^nodal_positions_(\d+?)_", column_name).group(1))

        # columns ending with "1" represent the x node
        if column_name.endswith("1"):
            x_nodes[node_num] = df.iloc[: , index]

        # columns ending with "2" represent the y node
        if column_name.endswith("2"):
            y_nodes[node_num] = df.iloc[: , index]

        # columns ending with "3" represent the z node
        if column_name.endswith("3"):
            z_nodes[node_num] = df.iloc[: , index]

    # Secondly, find the bar connections  using regex
    if re.search("^bar_connectivity", column_name):
        
        edge_num = int(re.search("^bar_connectivity_(\d+?)_", column_name).group(1))
        bar_connectivity[column_name] = df.iloc[: , index]

        # Find the node pairs and store them in respective DF
        if column_name.endswith("1"):
            first_nodes[edge_num] = df.iloc[: , index]
        if column_name.endswith("2"):
            second_nodes[edge_num] = df.iloc[: , index]

# replace the NAN values with 0
x_nodes = x_nodes.fillna(0)
y_nodes = y_nodes.fillna(0)
z_nodes = z_nodes.fillna(0)
first_nodes = first_nodes.fillna(0)
second_nodes = second_nodes.fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

Calculate the distace between atoms (nodes) 

In [84]:
import math

# create necessary empty dataframes
distances = pd.DataFrame()

xyz = pd.DataFrame()
imn = pd.DataFrame()

# Iterate the series of pairs of nodes concurently
for (index1, s1), (index2, s2) in zip(first_nodes.iterrows(), second_nodes.iterrows()):

    # iterate the column names hich represent the edge
    for edge_number in list(first_nodes.columns):
        # Find the two nodes connected to the particular edge
        node1 = int(s1[edge_number])
        node2 = int(s2[edge_number])

        try:
            x1 = x_nodes[node1][index1]
            x2 = x_nodes[node2][index2]

            y1 = y_nodes[node1][index1]
            y2 = y_nodes[node2][index2]

            z1 = z_nodes[node1][index1]
            z2 = z_nodes[node2][index2]

            xyz["xyz_{}".format(edge_number)]
            
            edge_length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            i = (x2 - x1) / edge_length
            m = (y2 - y1) / edge_length
            n = (z2 - z1) / edge_length
        except:
            edge_length = 0

            i, m, n = 0, 0, 0

        distances["dist_".format(edge_number)] = edge_length

        imn["imn_{}".format(edge_number)] = [i, m, n]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Compute node connectivity

In [85]:
node_connections = pd.DataFrame()
column_names = bar_connectivity.columns.values.tolist()

for name in column_names:
    edge_num = int(re.search("^bar_connectivity_(\d+?)_", name).group(1))
    node_connections["node_connectivity_{}".format(edge_num)] = bar_connectivity[bar_connectivity == edge_num].count(axis=1)

node_connections

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


,node_connectivity_1,node_connectivity_2,node_connectivity_3,node_connectivity_4,node_connectivity_5,node_connectivity_6,node_connectivity_7,node_connectivity_8,node_connectivity_9,node_connectivity_10,...,node_connectivity_94,node_connectivity_95,node_connectivity_96,node_connectivity_97,node_connectivity_98,node_connectivity_99,node_connectivity_100,node_connectivity_101,node_connectivity_102,node_connectivity_103
0,3,3,3,3,3,3,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,3,3,3,3,3,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,8,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,3,4,4,3,3,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2,2,2,3,2,3,2,3,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,2,2,3,6,6,3,2,6,6,...,0,0,0,0,0,0,0,0,0,0
1996,3,3,4,4,3,6,3,4,3,4,...,0,0,0,0,0,0,0,0,0,0
1997,1,2,1,6,1,2,1,1,4,3,...,0,0,0,0,0,0,0,0,0,0
1998,2,2,2,2,3,3,3,2,2,3,...,0,0,0,0,0,0,0,0,0,0


Extract a, b, c, alpha, beta, gamma

In [86]:
constant_features = df[["a", "b", "c", "alpha", "beta", "gamma"]]

Build a dataframe to contain all computed data

In [146]:
import json

model_data = pd.concat([node_connections, imn, constant_features], axis=1)
y_vector = df[["Cx", "Cy", "Cz", "nx", "ny", "nz"]]
model_data = model_data.to_json("multiclassfication.json", orient="index")

## Modeling

Install All necessary modules

In [ ]:
# !pip install pymatgen
# !pip install mendeleev 
# !pip install graphviz 
# !pip install pydot 
# !pip install sklearn
# !pip install crysnet

In [88]:
from crysnet.data import Dataset
from crysnet.data.generator import GraphGenerator
from crysnet.models import GNN
from crysnet.models.gnnmodel import CgcnnModel
from pymatgen.core.structure import Structure
from pathlib import Path

In [134]:
ModulePath = Path('/content/dataset_multiclassification.json')

In [147]:
dataset = Dataset(task_type='multiclassification', data_path=ModulePath)
BATCH_SIZE = 1600
DATA_SIZE = 2000
CUTOFF = 2.5

#structure the dataset for consumption by model
# dataset.prepare_x(model_data)
dataset.prepare_y(y_vector)

,Cx,Cy,Cz,nx,ny,nz
0,0.33,0.33,0.33,1.00,1.00,1.00
1,0.23,0.03,0.26,1.01,1.01,1.00
2,0.09,0.09,0.09,2.00,2.00,2.00
3,0.25,0.25,0.25,1.00,1.00,1.00
4,0.25,0.25,0.50,1.00,1.00,1.00
...,...,...,...,...,...,...
1995,0.41,0.16,0.25,1.22,1.83,1.05
1996,0.15,0.15,0.28,1.99,1.98,1.04
1997,0.24,0.13,0.09,1.01,1.01,1.01
1998,0.01,0.01,0.26,1.05,1.05,1.00


In [148]:
Generators = GraphGenerator(dataset, data_size=DATA_SIZE, batch_size=BATCH_SIZE, cutoff=CUTOFF)
train_data = Generators.train_generator
valid_data = Generators.valid_generator
test_data = Generators.test_generator
multiclassification = Generators.multiclassification  

TypeError: ignored

In [ ]:
gnn = GNN(model=CgcnnModel, atom_dim=16, bond_dim=64, num_atom=118, 
          state_dim=16, sp_dim=230, units=32, edge_steps=1, 
          message_steps=1, transform_steps=1, num_attention_heads=8, 
          dense_units=64, output_dim=64, readout_units=64, dropout=0.0, 
          reg0=0.00, reg1=0.00, reg2=0.00, reg3=0.00, reg_rec=0.00, batch_size=BATCH_SIZE, 
          spherical_harmonics=True, regression=dataset.regression, optimizer = 'Adam')

In [ ]:
try:
  gnn.train(train_data, valid_data, test_data, epochs=700, lr=3e-3, warm_up=True, load_weights=False, verbose=1, checkpoints=None, save_weights_only=True, workdir=ModulePath)
  gnn.predict_datas(test_data, workdir=ModulePath)    # predict on test datas with labels
except Exception as err:
  print(err)
y_pred_keras = gnn.predict(datas)

In [ ]:
import warnings
import logging
import time
from pathlib import Path
import numpy as np
import tensorflow as tf
from matdgl.data import Dataset
from matdgl.models import Finetune
from matdgl.data.generator import GraphGenerator
from matdgl.models.finetune import FinetuneTransformer, FinetuneTransformerRes

tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

np.random.seed(88)
tf.random.set_seed(88)

# get current work dir of test.py
ModulePath = Path(__file__).parent.absolute()

# read datas from ModulePath/datas/multiclassfication.json
print('reading dataset...')
'multiclassification'
'topology_multi'
'formation_energy'
'regression'
'my_regression'
start = time.time()
dataset = Dataset(task_type='topology_multi', data_path=ModulePath, ratio=[0.6, 0.8])
end = time.time()
run_time = end - start
print('done')
print('run time:  {:.2f} s'.format(run_time))
print(dataset.dataset_file)

BATCH_SIZE = 16
DATA_SIZE = None
CUTOFF = 4.5

# building batch generator for model trainning
Generators = GraphGenerator(dataset, data_size=DATA_SIZE, batch_size=BATCH_SIZE, cutoff=CUTOFF)
train_data = Generators.train_generator
valid_data = Generators.valid_generator
test_data = Generators.test_generator

#if task is multiclassfication, should define variable multiclassifiction
multiclassification = Generators.multiclassification 

epochs=32
lr=1e-3

# default trainning parameters
state_dim=16
sp_dim=230
output_dim=32
readout_units=128
dropout=0.0
reg2=0.0
reg3=0.0
reg_rec=0.0
regression=dataset.regression
optimizer = 'Adam'

print('\n----- parameters -----',
    '\ntask_type: ', dataset.task_type,
    '\nsample_size: ', Generators.data_size,
    '\ncutoff: ', CUTOFF,
    '\nstate_dim: ', state_dim,
    '\nsp_dim: ', sp_dim,
    '\noutput_dim: ', output_dim,
    '\nreadout_units: ', readout_units,
    '\ndropout: ', dropout,
    '\nreg2: ', reg2,
    '\nreg3: ', reg3,
    '\nreg_rec: ', reg_rec,
    '\noptimizer: ', optimizer,
    '\nmulticlassification: ', multiclassification,
    '\nregression: ', regression,)

del dataset

# default model is a GraphTransformer model, can be changed to MPNN model by set 'model=MpnnModel'
gnn = Finetune(model=FinetuneTransformerRes,
        state_dim=state_dim,
        sp_dim=sp_dim,
        output_dim=output_dim,
        readout_units=readout_units,
        dropout=dropout,
        reg2=reg2,
        reg3=reg3,
        reg_rec=reg_rec,
        optimizer = optimizer,
        regression=regression,
        multiclassification=multiclassification,
        )

# trainning model
gnn.train(train_data, valid_data, test_data, epochs=epochs, lr=lr, warm_up=True,\
             verbose=1, checkpoints=None, save_weights_only=True, workdir=ModulePath)

print('\n----- parameters -----',
    '\nsample_size: ', Generators.data_size,
    '\ncutoff: ', CUTOFF,
    '\nstate_dim: ', state_dim,
    '\nsp_dim: ', sp_dim,
    '\noutput_dim: ', output_dim,
    '\nreadout_units: ', readout_units,
    '\noptimizer: ', optimizer,
    '\nmulticlassification: ', multiclassification,
    '\nregression: ', regression,
    '\nepochs: ', epochs,
    '\nlr: ', lr)